# Simulated Effects Demo

This python notebook shows the simulated effects usage in the unified planning library.

## Setup the library and the planners

First, we install unified_planning library and its dependencies from PyPi. Here, we use the `--pre` flag to use the latest development build.

In [ ]:
# begin of installation

In [ ]:
pip install --pre unified-planning

     |████████████████████████████████| 166 kB 5.2 MB/s 


We download and install tamer

In [ ]:
!rm -rf up-tamer && git clone https://github.com/aiplan4eu/up-tamer && pip install up-tamer/

Cloning into 'up-tamer'...
remote: Enumerating objects: 407, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 407 (delta 84), reused 74 (delta 74), pack-reused 283
Receiving objects: 100% (407/407), 94.21 KiB | 1.92 MiB/s, done.
Resolving deltas: 100% (247/247), done.
Processing ./up-tamer
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for up-tamer: filename=up_tamer-0.0.1-py3-none-any.whl size=11819 sha256=75a5c88cba672e5b3ff97f7db4e6399bd5e19d54efd3e975401f3141bd687f22
  Stored in directory: /root/.cache/pip/wheels/4c/7d/54/0e6493c780d4154683fd4e9b06bfbe263b2

We are now ready to use the Unified-Planning library!

In [ ]:
# end of installation

## Demo


### Basic imports
The basic imports we need for this demo are abstracted in the `shortcuts` package.

In [ ]:
from unified_planning.shortcuts import *

### Problem definition

We start the problem modeling defining the `UserType` and the `Fluent`.

In [ ]:
Location = UserType('Location')
Robot = UserType('Robot')

at = Fluent('at', Location, robot=Robot)
battery_charge = Fluent('battery_charge', IntType(0, 100), robot=Robot)

We define an action `move` with a simulated effect that models the battery consumption.

A `SimulatedEffect` instance can affect a list of fluent expressions, in this case only `battery_charge(robot)`.
The function `fun` performs the computation of the simulated effect decreasing the battery value by 10. This function receives as parameters the problem, the state in which the effect is applied, and the actual parameters of the action instance whose effect is being calculated.


In [ ]:
move = InstantaneousAction('move', robot=Robot, l_from=Location, l_to=Location)
robot = move.parameter('robot')
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(Equals(at(robot), l_from))
move.add_precondition(GE(battery_charge(robot), 10))
move.add_precondition(Not(Equals(l_from, l_to)))
move.add_effect(at(robot), l_to)
def fun(problem, state, actual_params):
    value = state.get_value(battery_charge(actual_params.get(robot))).constant_value()
    return [Int(value - 10)]
move.set_simulated_effect(SimulatedEffect([battery_charge(robot)], fun))

Finally, we define the `Object` instances and, after creating the `Problem`, we set the initial values and the goal.

In [ ]:
l1 = Object('l1', Location)
l2 = Object('l2', Location)
r1 = Object('r1', Robot)

problem = Problem('robot_with_simulated_effects')
problem.add_fluent(at)
problem.add_fluent(battery_charge)
problem.add_action(move)
problem.add_object(l1)
problem.add_object(l2)
problem.add_object(r1)

problem.set_initial_value(at(r1), l1)
problem.set_initial_value(battery_charge(r1), 100)

problem.add_goal(Equals(at(r1), l2))

### Solving the problem

We solve the problem automatically selecting a suitable planner.

In [ ]:
with OneshotPlanner(problem_kind=problem.kind) as planner:
    result = planner.solve(problem)
    print("%s returned: %s" % (planner.name, result.plan))

Tamer returned: [move(r1, l1, l2)]
